## 해당 코드는 구글 코랩으로 작성됨.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

import re
import datetime
from datetime import datetime

import os
#os.chdir('/content/drive/MyDrive/랩실 프로젝트/iphone data')

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import pos_tag
from nltk import RegexpParser

nltk.download("punkt")
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import tensorflow as tf

# GPU 사용 설정
device_name = tf.test.gpu_device_name()
if not tf.config.list_physical_devices('GPU'):
    print("GPU를 찾을 수 없습니다. 런타임 유형을 GPU로 변경해주세요.")
else:
    print(f"GPU 장치: {device_name}")

GPU 장치: /device:GPU:0


# 1. 리뷰 데이터 병합

In [ ]:
#@title 전 데이터
df_sy = pd.read_csv('/content/drive/MyDrive/랩실 프로젝트/data/all_dataset_preprocessed.CSV', encoding='cp949')
df_sy = df_sy[['name', 'date', 'rate', 'title', 'content']].rename(columns={'name':'model', 'content':'content',
                                                                            'rate':'rating'})

def extract_date(date_string):
    if "00:00:00" in date_string:
        datetime_obj = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")
        return datetime_obj.strftime("%Y-%m-%d")
    else:
        return date_string

# 'date' 열에 함수 적용하여 날짜만 추출
df_sy['date'] = df_sy['date'].apply(extract_date)
df_sy.head()

,model,date,rating,title,content
0,galaxy_s3,2012-05-11,2,too plasticky for $700,When I put a review a while back people starte...
1,galaxy_s3,2012-06-01,5,So far super good,I got my S3 in white yesterday I had the iPhon...
2,galaxy_s3,2012-06-01,2,Expensive or Respect,i've narrowed my choices for my next cell phon...
3,galaxy_s3,2012-06-04,5,I like the fact to....,got my S3 in white I also own the iPhone 4s fa...
4,galaxy_s3,2012-06-06,5,One of the best phones this year.,For me the top phones of the year (currently J...


In [ ]:
#@title iphone 데이터

df_ip = pd.read_csv('/content/drive/MyDrive/랩실 프로젝트/data/raw data/iphone_total.csv') #, encoding='cp949')
ip_lst = ['iphone_xs', 'iphone_xs_max']
df_ip = df_ip[df_ip['model'].isin(ip_lst)]
df_ip = df_ip[['model', 'date', 'rating', 'title', 'context']].rename(columns={'context':'content'})
df_ip['rating'] = df_ip['rating'].astype(int)

df_ip.head()

,model,date,rating,title,content
0,iphone_xs,2023-05-12,5,Worth taking risk on refurbished,I wanted a newer model (Iohone x or higher ) b...
1,iphone_xs,2023-04-12,4,"Great Phone, Okay battery",So first off I didn’t receive the package unti...
2,iphone_xs,2023-05-12,5,Great price,My phone body is 9/10. There are a few blemis...
3,iphone_xs,2023-04-18,5,Very Satisfied with my Purchase,This phone was basically brand new when I rece...
4,iphone_xs,2023-05-10,5,Excellent Condition,"Phone is in excellent condition, the screen & ..."


In [ ]:
#@title galaxy 데이터

df_gp = pd.read_csv('/content/drive/MyDrive/랩실 프로젝트/data/raw data/Galaxy S.csv') #, encoding='cp949')
df_gp = df_gp[df_gp['model']=='Galaxy S10E']
df_gp = df_gp.replace('Galaxy S10E', 'galaxy_s10e')
df_gp = df_gp[['model', 'date', 'rating', 'title', 'context']].rename(columns={'context':'content'})


def convert_date_format(date_string):
    datetime_obj = pd.to_datetime(date_string)
    return datetime_obj.strftime('%Y-%m-%d')

# 'date' 열에 함수 적용하여 날짜 형태 변경
df_gp['date'] = df_gp['date'].apply(convert_date_format)
df_gp['rating'] = df_gp['rating'].astype(int)


df_gp.head()

,model,date,rating,title,content
7607,galaxy_s10e,2023-05-13,5,this phone is suberb,i love this s10e. it's it's a small phone smal...
7608,galaxy_s10e,2023-04-03,5,"have had for 6 months, still works great!",i've had this phone for 6 months now and still...
7609,galaxy_s10e,2023-05-16,4,nice,nice
7610,galaxy_s10e,2023-04-17,5,it is a really good phone for the price. not ...,i took the sim card from my old phone and put ...
7611,galaxy_s10e,2023-04-22,5,good phone for our toddler,needed a phone for our toddler. she's a type 1...


In [ ]:
df_total = pd.concat([df_sy, df_ip, df_gp])
df_total = df_total.drop_duplicates(subset=['model', 'title', 'content'], keep='first') # 중복 제거
df_total.insert(0, 'id', range(1, len(df_total) + 1))
df_total

,id,model,date,rating,title,content
0,1,galaxy_s3,2012-05-11,2,too plasticky for $700,When I put a review a while back people starte...
1,2,galaxy_s3,2012-06-01,5,So far super good,I got my S3 in white yesterday I had the iPhon...
2,3,galaxy_s3,2012-06-01,2,Expensive or Respect,i've narrowed my choices for my next cell phon...
3,4,galaxy_s3,2012-06-04,5,I like the fact to....,got my S3 in white I also own the iPhone 4s fa...
4,5,galaxy_s3,2012-06-06,5,One of the best phones this year.,For me the top phones of the year (currently J...
...,...,...,...,...,...,...
10435,76227,galaxy_s10e,2021-08-24,5,"upgraded from galaxy s10+. yeah, you read tha...","i loved my s10+, but the size and curved glass..."
10436,76228,galaxy_s10e,2023-04-20,5,the phone is amazing. this is just like the s9...,i had the s9 for 3 years and i didn't want to ...
10437,76229,galaxy_s10e,2023-03-16,5,muy bueno,muy buen producto
10438,76230,galaxy_s10e,2023-02-18,5,"el dispositivo llegó en muy buen estado, se no...",the media could not be loaded.


In [ ]:
df_total.to_csv('/content/drive/MyDrive/랩실 프로젝트/data/preprocessed data/all_dataset.csv', index=False)

# 2. 랜덤 추출
- 이후 검토에 사용

In [ ]:
import random
random_indices = random.sample(range(len(df_total)), 100)
random_df = df_total.iloc[random_indices]
random_df

,id,model,date,rating,title,content
9150,74718,galaxy_s10e,2020-07-08,5,great deal!,amazing phone the perfect size and can't tell ...
36852,36816,iphone5,2014-02-05,5,excellent seller,excellent phone brand new and sealed in its bo...
41934,41885,iphone6s,2016-11-24,1,One Star,the iphone6s is described as inactive but has ...
463,464,galaxy_s3,2012-11-12,1,Think twice before you buy this phone,I would love this phone if it works fine. I'm ...
8216,73787,galaxy_s10e,2022-10-01,5,i love this phone.,"i am an apple person, but wanted to switch. i ..."
...,...,...,...,...,...,...
2885,2886,galaxy_s3,2014-07-16,1,Water damaged phone being past as Refurbished,Very up set the phone had water damage a long ...
4980,4981,galaxy_s3,2015-11-14,5,Great phone for the price.,I love this phone. I know it isn't the most cu...
40071,40025,iphone5,2015-12-23,5,Its Apple Quality,I got thing to use with Ting awhile back. It i...
54395,54341,iphone7,2019-12-10,1,Poor Quality Phone,Phone did not work. Got very hot to the touch ...


In [ ]:
random_df.to_csv('/content/drive/MyDrive/랩실 프로젝트/data/preprocessed data/랜덤추출.csv', index=False)